In [1]:
import json
import csv
import os

In [32]:
def empty_csv_dict():
    return {
        'model': '',
        'filename': '',
        'model_params': '',
        'augmentation': None,
        'brightness': None,
        'precipitation': None,
        'surface': None,
        'weather': None,
        'ground_truth_objects': 0,
        'segmented_objects': 0,
        'matched_objects': 0,
        'precision': 0.0,
        'recall': 0.0,
        'f1': 0.0,
        'mean_iou': 0.0,
        'hardware': 'NVIDIA GeForce RTX 3060M, AMD Ryzen 7 5800H, 32 GB RAM'
    }

def segm_dict2csv_dict(segm_dict):
    csv_dict = empty_csv_dict()

    weather_options = [
        ["bright", "dark", "night"],
        ["none", "rainfall", "snowfall", "fog"],
        ['none', 'puddles', 'snow'],
        ["none", "sunny", "cloudy"]
    ]
    weather_fields = ['brightness', 'precipitation', 'surface', 'weather']

    # get weather options
    for k, options in zip(weather_fields, weather_options):
        for option in options:
            if option in segm_dict['weather']:
                csv_dict[k] = option
    
    # get model
    csv_dict['model'] = ' '.join(segm_dict['model'])
    # print(csv_dict['model'], 'fastsam' in csv_dict['model'] or 'yolo' in csv_dict['model'])
    if 'fastsam' in csv_dict['model'] or 'yolo' in csv_dict['model']:
        csv_dict['model'] = csv_dict['model'].replace('sam2.1 ', '')

    csv_dict['filename'] = segm_dict['filename']
    
    # get numerical fields
    for k in ['ground_truth_objects', 'segmented_objects', 'matched_objects', 'precision', 'recall', 'f1', 'mean_iou']:
        if k in segm_dict:
            csv_dict[k] = segm_dict[k]
        else:
            print(f"Warning: key={k} not in segm_dict({segm_dict['model']}, {segm_dict['filename']})")
            csv_dict[k] = 0

    return csv_dict

In [21]:
results_path = r"C:\Users\gtraw\OneDrive\Pulpit\UM sem. 2\ProjektBadawczy\apps\evaluation\results\all"

In [22]:
data = []
for f in os.listdir(results_path):
    with open(os.path.join(results_path, f), 'r') as json_file:
        json_data = json.load(json_file)
        if 'filename' not in json_data:
            json_data['filename'] = '.'.join(f.split('.')[:-1])
        data.append(json_data)

In [23]:
data[0]

{'weather': ['bright', 'cloudy'],
 'model': ['sam2.1', 'base+'],
 'time': 33.98261117935181,
 'ground_truth_objects': 14,
 'segmented_objects': 41,
 'matched_objects': 9,
 'precision': 0.21951219512195122,
 'recall': 0.6428571428571429,
 'f1': 0.32727272727272727,
 'mean_iou': 0.738776414364839,
 'filename': 'base+_weti1_2024_11_14_12_47'}

In [33]:
csv_dicts = [segm_dict2csv_dict(d) for d in data]

In [34]:
csv_dicts[400]

{'model': 'yolo11l',
 'filename': 'yolo11l_weti2_2024_11_14_14_29',
 'model_params': '',
 'augmentation': None,
 'brightness': 'bright',
 'precipitation': None,
 'surface': None,
 'weather': 'cloudy',
 'ground_truth_objects': 35,
 'segmented_objects': 23,
 'matched_objects': 20,
 'precision': 0.8695652173913043,
 'recall': 0.5714285714285714,
 'f1': 0.689655172413793,
 'mean_iou': 0.6364619662386247,
 'hardware': 'NVIDIA GeForce RTX 3060M, AMD Ryzen 7 5800H, 32 GB RAM'}

In [35]:
keys = csv_dicts[0].keys()
output_path = r"C:\Users\gtraw\OneDrive\Pulpit\UM sem. 2\ProjektBadawczy\apps\evaluation\csvs\2024_11_14.csv"

with open(output_path, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(csv_dicts)

In [8]:
import pandas as pd

df = pd.read_csv(output_path)
print(df.describe())
print(df.head)

       augmentation  precipitation  surface  ground_truth_objects  \
count           0.0            0.0      0.0            660.000000   
mean            NaN            NaN      NaN             17.733333   
std             NaN            NaN      NaN             13.022513   
min             NaN            NaN      NaN              0.000000   
25%             NaN            NaN      NaN              9.000000   
50%             NaN            NaN      NaN             13.000000   
75%             NaN            NaN      NaN             30.000000   
max             NaN            NaN      NaN             40.000000   

       segmented_objects  matched_objects   precision      recall          f1  \
count         660.000000       660.000000  660.000000  660.000000  660.000000   
mean           40.212121        12.593939    0.474604    0.546361    0.435722   
std            65.630759        11.891082    0.370169    0.346649    0.295997   
min             0.000000         0.000000    0.000000 

In [45]:
for i, m in enumerate(df['model'].unique()):
    print(f"{i+1:<3} {m}")

1   sam2.1 base+
2   sam2.1 fastsam_x
3   sam2.1 large
4   sam2.1 small
5   sam2.1 tiny
6   sam2.1 fastsam_s
7   sam2.1 yolo11l
8   sam2.1 yolo11m
9   sam2.1 yolo11n
10  sam2.1 yolo11s
11  sam2.1 yolo11x
